<a href="https://colab.research.google.com/github/cvillarr123/trading/blob/master/Notebook%5CForexRoberto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install alpha_vantage finnhub-python pandas

In [ ]:
import pandas as pd 
import json
from pprint import pprint
from datetime import datetime
import dateutil.parser
from pytz import timezone
from dateutil import tz
from alpha_vantage.foreignexchange import ForeignExchange
import finnhub
# Setup client
alpha_client = ForeignExchange(key='RCMEEIVUZYDWGCDH')
finnhub_client = finnhub.Client(api_key="c2v8lvaad3i9mrpv2ru0")

In [ ]:
def convert_date_to_unix(p_date):
  gmt5 = timezone('America/Guayaquil')
  naive_ts = datetime.strptime(p_date, '%Y-%m-%d %H:%M:%S')
  local_ts = gmt5.localize(naive_ts)
  epoch_ts = local_ts.timestamp()
  final_date = str(epoch_ts).split(".")[0] 
  return int(final_date)

In [ ]:
def fx_homologate(source, function, currency_from, currency_to, type_exchange, period, date_ini, date_end):
  data = ""
  if function == 'EXCHANGE':
    if source == 'ALPHA':
      res, _ = alpha_client.get_currency_exchange_rate(from_currency=currency_from,to_currency=currency_to)
      data = pd.DataFrame([res])
      data.columns=['From_Currency_Code','From_Currency_Name', 'To_Currency_Code', 'To_Currency_Name',
                    'Exchange_Rate', 'Last_Refreshed', 'Time_Zone', 'Bid_Price', 'Ask_Price']
    elif source == 'FINNHUB':
      res = finnhub_client.forex_rates(base=currency_from)
      exchange = res['quote']
      xe = pd.DataFrame([exchange])
      format_alpha = {'From_Currency_Code':currency_from,'From_Currency_Name':'',
              'To_Currency_Code':currency_to, 'To_Currency_Name':'',
             'Exchange_Rate':xe[currency_to], 'Last_Refreshed': datetime.utcnow(),
              'Time_Zone':'UTC', 'Bid_Price':'', 'Ask_Price':''}
      data = pd.DataFrame(format_alpha)
  elif function == 'CANDLES':
    if period == '1' or period == '5' or period == '15' or period == '30' or period == '60':
      if source == 'ALPHA':
        period = period+'min'
        data = alpha_client.get_currency_exchange_intraday(from_symbol=currency_from, to_symbol=currency_to, interval=period)
      elif source == 'FINNHUB':
        d = finnhub_client.forex_symbols(type_exchange)
        df = pd.DataFrame(d)
        record = df[df['displaySymbol'].str.match(currency_from+'/'+currency_to)]
        res = finnhub_client.forex_candles(record.symbol, period, convert_date_to_unix(date_ini), convert_date_to_unix(date_end))
        if res["s"] == 'no_data':
          data = pd.DataFrame.from_dict(res, orient="index")
        else:
          data = pd.DataFrame(res)
    elif period == 'D' or period == 'W' or period == 'M':
      if source == 'ALPHA':
        if period == 'D':
          data = alpha_client.get_currency_exchange_daily(from_symbol=currency_from, to_symbol=currency_to)
        if period == 'W':
          data = alpha_client.get_currency_exchange_weekly(from_symbol=currency_from, to_symbol=currency_to)
        if period == 'M':
          data = alpha_client.get_currency_exchange_monthly(from_symbol=currency_from, to_symbol=currency_to)
      elif source == 'FINNHUB':
        d = finnhub_client.forex_symbols(type_exchange)
        df = pd.DataFrame(d)
        record = df[df['displaySymbol'].str.match(currency_from+'/'+currency_to)]
        res = finnhub_client.forex_candles(record.symbol, period, convert_date_to_unix(date_ini), convert_date_to_unix(date_end))
        if res["s"] == 'no_data':
          data = pd.DataFrame.from_dict(res, orient="index")
        else:
          data = pd.DataFrame(res)
  return data

In [ ]:
#pprint(fx_homologate('FINNHUB', 'EXCHANGE','GBP', 'AUD', 'W'))
fe = fx_homologate('FINNHUB', 'CANDLES', 'GBP', 'AUD', 'pepperstone', 'W', '2021-07-10 00:00:00', '2021-07-12 00:00:00')
fe

,c,h,l,o,s,t,v
0,1.853175,1.86093,1.85049,1.85477,ok,1626037200,310529


In [ ]:
convert_date_to_unix("2021-07-12 22:29:59")


1626146999